In [5]:
# Data Analysis #
import pandas as pd 
import numpy as np
import re

# Graphics #
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas
#from shapely.geometry import Point
%matplotlib inline

# Display up to 60 columns of a dataframe
pd.set_option('display.max_columns', 60)

# Set default font size
plt.rcParams['font.size'] = 24

# ML packages #

# Splitting data into training and testing
from sklearn.model_selection import train_test_split

In [7]:
# Data #
tag = 'Healthy_Aging_Data1.csv'
raw_df = pd.read_csv(tag)
raw_df.sample(5)

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Response,Data_Value_Unit,DataValueTypeID,Data_Value_Type,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,StratificationCategory3,Stratification3,GeoLocation,ClassID,TopicID,QuestionID,ResponseID,LocationID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Report
1053,2014,2014,SD,South Dakota,BRFSS,Overall Health,Oral health: tooth retention,Percentage of older adults who report having l...,NaN,%,PRCTG,Percentage,60.9,NaN,NaN,NaN,57.5,64.1,NaN,Age Group,65 years or older,NaN,NaN,NaN,NaN,"(44.353130053000484, -100.3735306369997)",C01,TOC05,Q07,NaN,46,AGE,65PLUS,NaN,NaN,NaN,NaN,NaN
9208,2013,2013,HI,Hawaii,BRFSS,Screenings and Vaccines,Influenza vaccine within past year,Percentage of older adults who reported influe...,NaN,%,PRCTG,Percentage,51.6,NaN,NaN,NaN,46.6,56.6,NaN,Age Group,60-64 years,NaN,NaN,NaN,NaN,"(21.304850435000446, -157.85774940299973)",C03,TSC08,Q18,NaN,15,AGE,6064,NaN,NaN,NaN,NaN,NaN
13314,2012,2012,SC,South Carolina,BRFSS,Overall Health,Physically unhealthy days (mean number of days),Physically unhealthy days (mean number of days...,NaN,Number,MEAN,Mean,6.6,NaN,NaN,NaN,5.8,7.5,NaN,Age Group,55-59 years,NaN,NaN,NaN,NaN,"(33.998821303000454, -81.04537120699968)",C01,TOC01,Q08,NaN,45,AGE,5559,NaN,NaN,NaN,NaN,NaN
34602,2016,2016,AZ,Arizona,BRFSS,Nutrition/Physical Activity/Obesity,No leisure time physical activity within past ...,Percentage of older adults who have not had an...,NaN,%,PRCTG,Percentage,28.2,NaN,NaN,NaN,24.8,32.0,NaN,Age Group,60-64 years,NaN,NaN,NaN,NaN,"(34.865970280000454, -111.76381127699972)",C02,TNC03,Q16,NaN,4,AGE,6064,NaN,NaN,NaN,NaN,NaN
11742,2012,2012,SC,South Carolina,BRFSS,Nutrition/Physical Activity/Obesity,Eating 2 or more fruits daily,Percentage of older adults who are eating 2 or...,NaN,%,PRCTG,Percentage,NaN,NaN,~,No Data Available,NaN,NaN,NaN,Age Group,50-54 years,NaN,NaN,NaN,NaN,"(33.998821303000454, -81.04537120699968)",C02,TNC01,Q01,NaN,45,AGE,5054,NaN,NaN,NaN,NaN,NaN


In [8]:
filt_cognitive = raw_df['Class'] == 'Cognitive Decline'
cognitive_df = raw_df[filt_cognitive]

In [9]:
def drop_empty_columns(df):
    ''' Drop Empty Columns From A DataFrame'''
    all_empty = df.shape[0]
    to_be_dropped = [cols for cols in df.columns if df[cols].isnull().sum() == all_empty ]
    print(to_be_dropped)
    df = df.drop(columns=to_be_dropped)
    return df

In [14]:
cognitive_df = drop_empty_columns(cognitive_df)
check_units = [col for col in cognitive_df if col.startswith('Data')]
if 'Data_Value' in check_units:
    check_units.remove('Data_Value')
    cognitive_df = cognitive_df.drop(columns=check_units)


[]


In [15]:
to_drop = ['GeoLocation',
          'StratificationID1',
          'QuestionID',
          'LocationID',
          'ClassID',
          'TopicID',
          'LocationDesc',
          'StratificationCategoryID1',
          'StratificationCategoryID2',
          'StratificationID2',
          'StratificationCategory1',
          'StratificationCategory2'
          ]
cognitive_df_clean = cognitive_df.drop(columns=to_drop)

In [16]:
def missing_values_table(df):
    '''Calculate Missing Values of Each Column'''
    # Total missing amount in each feature /series type
    missing_value = df.isnull().sum()

    # Percentage of that missing values in its column /series type
    miss_percent = 100*missing_value/df.shape[0]

    # Make a table of calculated fields /will make dataframe
    table = pd.concat([missing_value, miss_percent], axis=1).rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort by percentages column
    table.sort_values(by=['% of Total Values'], ascending=False,
                      inplace=True)

    # Print Summary Information
    print('\nThe DataFrame has ' + str(df.shape[1]) + ' columns.\n'
          'There are ' + str(table.shape[0]) + ' columns that have missing values')

    return table

missing_values_table(cognitive_df_clean)


The DataFrame has 11 columns.
There are 11 columns that have missing values


,Missing Values,% of Total Values
Stratification2,3304,89.393939
Data_Value,1618,43.777056
Low_Confidence_Limit,1618,43.777056
High_Confidence_Limit,1618,43.777056
YearStart,0,0.000000
YearEnd,0,0.000000
LocationAbbr,0,0.000000
Class,0,0.000000
Topic,0,0.000000
Question,0,0.000000


In [17]:
missing_df = missing_values_table(cognitive_df_clean)
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50.0].index)
print('We will remove {} column(s).'.format(missing_columns))


The DataFrame has 11 columns.
There are 11 columns that have missing values
We will remove ['Stratification2'] column(s).


In [18]:
cognitive_df_clean = cognitive_df_clean.drop(columns=list(missing_columns))

# This is the dataframe I am working on


In [20]:
cognitive_df_clean[['LocationAbbr', 'YearStart','Stratification1', 'Data_Value']]

,LocationAbbr,YearStart,Stratification1,Data_Value
21919,US,2011,50-54 years,10.2
21920,US,2011,50-54 years,9.0
21921,US,2011,50-54 years,8.0
21922,US,2011,50-54 years,9.8
21923,US,2011,50-54 years,10.7
21924,US,2011,50-54 years,26.2
21925,US,2011,50-54 years,9.2
21926,AL,2011,50-54 years,NaN
21927,AK,2011,50-54 years,NaN
21928,AZ,2011,50-54 years,NaN


 **I want to fill missing values in Data_Value column taking mean of sub group it is in. i.e. If State is AZ and Year 2015 and 50-54 year old then take the mean of Data_Value and fill nan in this small group**.
 
 Code below takes infinite time to do work. I actually did not get anything out of it. Could you recommend me soem other ways?

In [21]:
for i in cognitive_df_clean.LocationAbbr:
    for j in cognitive_df_clean.YearStart:
        for k in cognitive_df_clean.Stratification1:
            age_grp = cognitive_df_clean['Stratification1'] == k
            state = cognitive_df_clean['LocationAbbr'] == i 
            year = cognitive_df_clean['YearStart'] == j 
            avg = cognitive_df_clean.loc[state  & age_grp, 'Data_Value'].mean()
            cognitive_df_clean.loc[state & year & age_grp, 'Data_Value'].fillna(avg)
        

NameError: name 'year' is not defined